In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным с примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Статистика по типам данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропущенные значения
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing/df.shape[0]*100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, 
                        keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры данных
    print(f"\nПример первых {sample_size} записей:")
    display(df.head(sample_size))

def load_data(data_path: Path) -> tuple:
    """Загрузка данных из PKL-файлов"""
    try:
        sessions = pd.read_pickle(data_path / 'ga_sessions.pkl')
        hits = pd.read_pickle(data_path / 'ga_hits.pkl')
        return sessions, hits
    except Exception as e:
        raise RuntimeError(f"Ошибка загрузки данных: {str(e)}")

def optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Оптимизация типов данных"""
    # Категориальные признаки
    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        num_unique = df[col].nunique()
        if num_unique < df.shape[0] * 0.5:
            df[col] = df[col].astype('category')
            
    # Числовые признаки
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], downcast='integer', errors='ignore')
        
    return df

def process_datetime(df: pd.DataFrame, date_col: str, time_col: str) -> pd.DataFrame:
    """Обработка временных меток с валидацией"""
    df = df.copy()
    
    # Обработка даты
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    # Обработка времени
    if time_col in df.columns:
        # Конвертация числовых значений времени (например, 143050 -> 14:30:50)
        df[time_col] = df[time_col].apply(
            lambda x: f"{int(x//10000):02d}:{int((x%10000)//100):02d}:{int(x%100):02d}" 
            if isinstance(x, (int, float)) and not pd.isna(x) else x
        )
        
        # Исправление некорректных значений
        df[time_col] = df[time_col].replace({
            '24:00:00': '00:00:00',
            '2400': '00:00:00',
            'nan': pd.NaT
        })
        
        # Конвертация в datetime.time
        df[time_col] = pd.to_datetime(
            df[time_col], format='%H:%M:%S', errors='coerce'
        ).dt.time
    
    # Создание полной метки времени
    if date_col in df.columns and time_col in df.columns:
        df['datetime'] = pd.to_datetime(
            df[date_col].astype(str) + ' ' + df[time_col].astype(str),
            errors='coerce'
        )
        # Фильтрация некорректных дат
        df = df[df['datetime'].dt.year.between(2020, 2023)]
    
    return df

def handle_device_data(df: pd.DataFrame) -> pd.DataFrame:
    # Заполняем NaN единообразным значением
    df = df.fillna({
        'device_os': 'unknown',
        'device_brand': 'unknown',
        'device_model': 'generic',
        'device_browser': 'unknown',
        'device_screen_resolution': '0x0'
    })

    # Заменяем любые строки, не совпадающие с шаблоном «\d+x\d+», на '0x0'
    mask = df['device_screen_resolution'].str.match(r'^\d+x\d+$', na=False)
    df.loc[~mask, 'device_screen_resolution'] = '0x0'

    # Теперь безопасно разбиваем и приводим к числу
    df[['screen_width', 'screen_height']] = (
        df['device_screen_resolution']
          .str.split('x', expand=True)
          .astype(int)
    )
    df = df.drop('device_screen_resolution', axis=1)
    return df

def process_utm(df: pd.DataFrame) -> pd.DataFrame:
    """Обработка UTM-меток"""
    utm_map = {
        'utm_source': {
            '(none)': 'organic',
            '(not set)': 'organic',
            'nan': 'organic'
        },
        'utm_medium': {
            '(none)': 'organic',
            'not set': 'organic',
            'nan': 'organic'
        },
        'utm_campaign': {
            '(not set)': 'general',
            'nan': 'general'
        },
        'utm_adcontent': {
            '(not set)': 'none',
            'nan': 'none'
        },
        'utm_keyword': {
            '(not set)': 'none',
            'nan': 'none'
        }
    }
    
    for col, mapping in utm_map.items():
        if col in df.columns:
            df[col] = (
                df[col]
                .replace(mapping)
                .fillna('other')
                .astype('category')
            )
    return df

def add_conversion_target(sessions: pd.DataFrame, hits: pd.DataFrame) -> pd.DataFrame:
    """Добавление целевой переменной (конверсия)"""
    # Определение целевых действий
    target_actions = [
        'sub_button_click', 
        'sub_page_view', 
        'sub_view_cars_click',
        'sub_landing'
    ]
    
    # Создание флага конверсии
    hits['is_conversion'] = hits['event_action'].isin(target_actions).astype(int)
    
    # Агрегация по сессиям
    conversions = hits.groupby('session_id')['is_conversion'].max().reset_index()
    
    # Слияние с сессиями
    sessions = sessions.merge(
        conversions,
        on='session_id',
        how='left'
    ).fillna({'is_conversion': 0})
    
    return sessions

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Очистка данных"""
    # Удаление дубликатов
    df = df.drop_duplicates()
    
    # Удаление полностью пустых колонок
    df = df.dropna(axis=1, how='all')
    
    # Удаление некорректных записей
    df = df[df['session_id'].notna()]
    
    return df

def main_pipeline(data_path: Path, save_path: Path) -> None:
    """Полный пайплайн обработки данных"""
    # Загрузка данных
    sessions, hits = load_data(data_path)
    
    # Вывод примеров исходных данных
    print("\n" + "="*40 + " ИСХОДНЫЕ ДАННЫЕ " + "="*40)
    print("\nСЕССИИ ДО ОБРАБОТКИ:")
    display(sessions.head(3))
    generate_summary(sessions, "Исходные сессии")
    
    print("\nСОБЫТИЯ ДО ОБРАБОТКИ:")
    display(hits.head(3))
    generate_summary(hits, "Исходные события")

    # Обработка сессий
    sessions_processed = (
        sessions.pipe(clean_data)
        .pipe(process_datetime, 'visit_date', 'visit_time')
        .pipe(handle_device_data)
        .pipe(process_utm)
        .pipe(optimize_dtypes)
    )
    
    # Обработка событий
    hits_processed = (
        hits.pipe(clean_data)
        .pipe(process_datetime, 'hit_date', 'hit_time')
        .pipe(optimize_dtypes)
    )
    
    # Добавление целевой переменной
    sessions_processed = add_conversion_target(sessions_processed, hits_processed)
    
    # Вывод примеров обработанных данных
    print("\n" + "="*40 + " ОБРАБОТАННЫЕ ДАННЫЕ " + "="*40)
    print("\nСЕССИИ ПОСЛЕ ОБРАБОТКИ:")
    display(sessions_processed.head(3))
    generate_summary(sessions_processed, "Обработанные сессии")
    
    print("\nСОБЫТИЯ ПОСЛЕ ОБРАБОТКИ:")
    display(hits_processed.head(3))
    generate_summary(hits_processed, "Обработанные события")
    
    # Сохранение
    save_path.mkdir(exist_ok=True)
    sessions_processed.to_pickle(save_path / 'processed_sessions.pkl')
    hits_processed.to_pickle(save_path / 'processed_hits.pkl')
    print("\n✅ Данные успешно обработаны и сохранены!")

if __name__ == "__main__":
    # Настройка путей через parent
    current_dir = Path(os.getcwd()).absolute()
    root_dir = current_dir.parent.parent  # Поднимаемся на уровень выше
    data_path = root_dir / 'data' / 'raw_data'
    save_path = root_dir / 'data' / 'processed_data'
    
    # Запуск пайплайна
    main_pipeline(data_path, save_path)


======================================== ИСХОДНЫЕ ДАННЫЕ ========================================

СЕССИИ ДО ОБРАБОТКИ:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk



================================================== ИСХОДНЫЕ СЕССИИ ==================================================
Общее количество записей: 1,860,042
Количество признаков: 18

Типы данных:
        count
object     17
int64       1

Пропущенные значения:
                 count      %
device_model   1843704  99.12
utm_keyword    1082061  58.17
device_os      1070138  57.53
utm_adcontent   335615  18.04
utm_campaign    219603  11.81
device_brand    118678   6.38
utm_source          97   0.01

Дубликаты: 0 (0.00%)

Пример первых 3 записей:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk



СОБЫТИЯ ДО ОБРАБОТКИ:


...


СОБЫТИЯ ПОСЛЕ ОБРАБОТКИ:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,datetime,is_conversion
9,2692901778487480807.1640206845.1640206845,2021-12-23,00:00:00,1,event,NaN,sberauto.com/cars/all/nissan/x-trail/0744675f?...,card_web,view_card,NaN,2021-12-23 00:00:00,0
11,4555345648396008371.1640233907.1640233907,2021-12-23,00:01:07,4,event,trXmoxcphNjuYcShOXwL,sberauto.com/cars/all/nissan/qashqai/bfc21661?...,card_web,view_card,NaN,2021-12-23 00:01:07,0
12,5636135307570360848.1640280592.1640280592,2021-12-23,00:00:12,3,event,NaN,sberauto.com/cars/all/renault/duster/145d53f1?...,card_web,view_card,NaN,2021-12-23 00:00:12,0



================================================== ОБРАБОТАННЫЕ СОБЫТИЯ ==================================================
Общее количество записей: 2,184,402
Количество признаков: 12

Типы данных:
                count
datetime64[ns]      2
category            1
category            1
int16               1
category            1
category            1
category            1
category            1
category            1
category            1
int64               1

Пропущенные значения:
               count      %
hit_referer  2073556  94.93
event_label  1355292  62.04

Дубликаты: 0 (0.00%)

Пример первых 3 записей:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,datetime,is_conversion
9,2692901778487480807.1640206845.1640206845,2021-12-23,00:00:00,1,event,NaN,sberauto.com/cars/all/nissan/x-trail/0744675f?...,card_web,view_card,NaN,2021-12-23 00:00:00,0
11,4555345648396008371.1640233907.1640233907,2021-12-23,00:01:07,4,event,trXmoxcphNjuYcShOXwL,sberauto.com/cars/all/nissan/qashqai/bfc21661?...,card_web,view_card,NaN,2021-12-23 00:01:07,0
12,5636135307570360848.1640280592.1640280592,2021-12-23,00:00:12,3,event,NaN,sberauto.com/cars/all/renault/duster/145d53f1?...,card_web,view_card,NaN,2021-12-23 00:00:12,0



✅ Данные успешно обработаны и сохранены!
